잡음 제거 테스트

In [9]:
import speech_recognition as sr
import wave
import io
import noisereduce as nr

# 오디오 데이터를 wav 파일로 저장하는 함수
def save_buffer_to_wav(buffer, sample_rate, sample_width, filename):
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(1)  # 모노
        wf.setsampwidth(sample_width)
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(buffer))

# 마이크에서 음성을 스트리밍하는 함수
def record_audio(filename="output.wav"):
    recognizer = sr.Recognizer()
    microphone = sr.Microphone(sample_rate=16000)
    buffer = []
    chunk_size = 16000
    num_chunks = 5

    with microphone as source:
        print("Adjusting for ambient noise..."); recognizer.adjust_for_ambient_noise(source)  # 주변 소음을 기준으로 에너지 임계값 조정
        
        print("Recording...")
        recognizer.energy_threshold = recognizer.energy_threshold + 100

        while len(buffer) < num_chunks:
            buffer.append(source.stream.read(chunk_size))

    sample_rate = microphone.SAMPLE_RATE
    sample_width = microphone.SAMPLE_WIDTH
    save_buffer_to_wav(buffer, sample_rate, sample_width, filename)
    print("Audio saved to {}".format(filename))

if __name__ == "__main__":
    record_audio()


Adjusting for ambient noise...
Recording...
Audio saved to output.wav


In [ ]:
'''
240620 sk 사물인식 건으로 음성인식 업무 보류.
noisereduce 라이브러리 이용해서 노이즈 감소시키는 테스트 해보려고 함.
아래 코드는 chatgpt가 마지막으로 만들어준 코드
바로 위 코드에다가 노이즈 감소 코드를 추가했고 아직 실행은 안해봄
'''

import speech_recognition as sr
import wave
import noisereduce as nr
import numpy as np

# 오디오 데이터를 wav 파일로 저장하는 함수
def save_buffer_to_wav(buffer, sample_rate, sample_width, filename):
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(1)  # 모노
        wf.setsampwidth(sample_width)
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(buffer))

# 마이크에서 음성을 스트리밍하는 함수
def record_audio(filename="output.wav"):
    recognizer = sr.Recognizer()
    microphone = sr.Microphone(sample_rate=16000, chunk_size=1024)
    buffer = []
    chunk_size = 1024
    num_chunks = int(5 * microphone.SAMPLE_RATE / chunk_size)

    with microphone as source:
        print("Adjusting for ambient noise...")
        recognizer.adjust_for_ambient_noise(source)  # 주변 소음을 기준으로 에너지 임계값 조정

        print("Recording...")
        recognizer.energy_threshold += 100

        while len(buffer) < num_chunks:
            buffer.append(source.stream.read(chunk_size))

    # 버퍼를 하나의 오디오 데이터로 결합
    audio_data = np.frombuffer(b''.join(buffer), dtype=np.int16)
    # 노이즈 감소
    reduced_noise = nr.reduce_noise(y=audio_data, sr=microphone.SAMPLE_RATE)

    # 노이즈 감소된 데이터를 다시 버퍼로 변환
    buffer = [reduced_noise.tobytes()]

    sample_rate = microphone.SAMPLE_RATE
    sample_width = microphone.SAMPLE_WIDTH
    save_buffer_to_wav(buffer, sample_rate, sample_width, filename)
    print("Audio saved to {}".format(filename))

if __name__ == "__main__":
    record_audio()
